In [1]:
!pip install koalas
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.2/593.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=9fe3d9d01de93c2f4f76ac68c9d92ad847d9b51efc50b3c97c38a7af5f3906fd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [8]:
import pandas as pd
import pyspark.pandas as ks
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pyspark.pandas as ks
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

! wget https://storage.googleapis.com/bdt-spark-store/external_sources.csv -O gcs_external_sources.csv
! wget https://storage.googleapis.com/bdt-spark-store/internal_data.csv -O gcs_internal_data.csv

df_data = ks.read_csv("gcs_internal_data.csv", header=True, inferSchema=True)
df_ext = ks.read_csv("gcs_external_sources.csv", header=True, inferSchema=True)

df_full = df_data.merge(df_ext, on='SK_ID_CURR', how='inner')

categorical_columns = ['NAME_EDUCATION_TYPE', 'CODE_GENDER', 'ORGANIZATION_TYPE', 'NAME_INCOME_TYPE']
for col in categorical_columns:
    df_full[col + "_index"] = df_full[col].cat.codes
    df_full[col + "_encoded"] = ks.get_dummies(df_full[col + "_index"])

input_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
               'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
               'AMT_ANNUITY', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
               'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE'] + \
              [col + "_encoded" for col in categorical_columns]

df_full["features"] = df_full[input_cols]

train = df_full.sample(frac=0.8, random_state=seed)
test = df_full.sample(frac=0.2, random_state=seed)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

random_forest = RandomForestClassifier(n_estimators=100, random_state=50)
random_forest.fit(train["features"], train["TARGET"])
predictions = random_forest.predict(test["features"])

accuracy = accuracy_score(test["TARGET"], predictions)
print("Accuracy:", (accuracy * 100))

model.save("random_forest_model")
ks.reset_option("compute.default_index_type")
ks.stop()




--2023-11-06 14:42:17--  https://storage.googleapis.com/bdt-spark-store/external_sources.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.207, 173.194.203.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15503836 (15M) [text/csv]
Saving to: ‘gcs_external_sources.csv’

gcs_external_source 100%[===================>]  14.79M  10.9MB/s    in 1.4s    

2023-11-06 14:42:18 (10.9 MB/s) - ‘gcs_external_sources.csv’ saved [15503836/15503836]

--2023-11-06 14:42:18--  https://storage.googleapis.com/bdt-spark-store/internal_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.207, 173.194.203.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152978396 (146M) [text/csv]
Saving to: ‘gcs_interna

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas Series is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


ValueError: ignored